In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from DataPreprocessor import DataPreprocessor 
from TrainTestPipeline import TrainTestPipeline
import pandas as pd

IndentationError: unindent does not match any outer indentation level (data_preprocessing.py, line 274)

In [ ]:
data_folder = '../RawData'
flow_values_excel = '../TrueFlowValues_.xlsx'
dp = DataPreprocessor(data_folder, flow_values_excel)

In [ ]:
success = dp.prepare_df()
assert success

In [ ]:
import numpy as np

In [ ]:
dp.make_histograms()

In [ ]:
data = dp.get_data()
for col in data.columns:
    print(data.loc[:,col].var())

LDA Visualizations

1. No Shrinkage with SVD solver

In [ ]:
data = dp.get_data().columns
data

In [ ]:
heldout_cols =  ["MinSize", "MaxSize", "Density"]
lda = dp.get_lda_x(solver='svd', shrinkage=None, heldout_cols = heldout_cols)
lda

In [ ]:
y_regr = dp.get_classification_y()
dp.visualize_lda(transformed_data = lda, targs = y_regr, 
                 out_file_name = "LDA_Microtrac.pdf")

2. Shrinkage with Eigen Solver

In [3]:
lda = dp.get_lda_x(solver='eigen', heldout_cols = heldout_cols)

NameError: name 'dp' is not defined

In [ ]:
y_regr = dp.get_classification_y()
dp.visualize_lda(transformed_data = lda, targs = y_regr, 
                 out_file_name = "LDA_Microtrac.pdf")

3. No shrinkage with eigen solver

In [ ]:
lda = dp.get_lda_x(solver='eigen', shrinkage=None, heldout_cols = heldout_cols)

In [ ]:
y_regr = dp.get_classification_y()
dp.visualize_lda(transformed_data = lda, targs = y_regr, 
                 out_file_name = "LDA_Microtrac.pdf")

In [ ]:
lda = dp.get_lda_x_fit(heldout_cols = ["MinSize", "MaxSize", "Density"])
prediction = lda.predict(dp.get_data(heldout_cols=['MinSize', 'MaxSize', 'Density']))
prediction

In [ ]:
real = dp.get_classification_y()
count = 0

for i in range(len(prediction)):
    if prediction[i] == real[i]:
        count += 1
percentage = count / len(prediction)
percentage

In [ ]:
qda = dp.get_qda_x(heldout_cols=['MinSize', 'MaxSize', 'Density'])
qda

In [ ]:
dp.get_data(heldout_cols=['MinSize', 'MaxSize', 'Density'])

In [ ]:
predict = qda.predict(dp.get_data(heldout_cols=['MinSize', 'MaxSize', 'Density']))
predict

In [ ]:
all_zero = True
for p in predict:
    if p != 0:
        all_zero = False
all_zero
        

In [ ]:
real = dp.get_classification_y()
count = 0

for i in range(len(predict)):
    if predict[i] == real[i]:
        count += 1
percentage = count / len(predict)
percentage

In [ ]:
data = dp.get_data(heldout_cols=['MinSize', 'MaxSize', 'Density'])

In [ ]:
all_samples = dp.get_samples()
# DecisionTreeRegressor trained on the best train set for augmented data 
# filtered by Pearson correlation
pipeline = TrainTestPipeline(x_data = data, y_data = y_regr, all_samples = all_samples, 
                             model_name = 'QDAAnalysis', 
                             heldout_samples = ['SS-17-4 (10-45)', 
                                                'SS-17-4 (64-150)', 
                                                'Ti-5Al-5V-5Mo-3Cr (0-75)'])

In [ ]:
tr_test_ = pipeline.do_train_test(cv = True)

In [ ]:
#visualize results
model_ = tr_test_['model']
data_split = tr_test_['train_test_split']
pipeline.visualize_performance(model = model_, train_test_split = data_split, 
                               output_file = 'QDAClassifyViz.pdf')